In [1]:
import glob
import re
import pandas as pd
import numpy as np
import copy
import calendar

files_txt = glob.glob(r'images/[0-9]*/*/*[.txt|.TXT]')
mtl_regex = re.compile('(.*/.*MTL.*)')

files_metadata = []
for file_path in files_txt:
    matched = mtl_regex.match(file_path)
    if matched is not None:
        files_metadata.append(matched.group())

In [2]:
files_metadata

['images/1/EO1A2260862014101110K2_1T/EO1A2260862014101110K2_MTL_L1T.TXT',
 'images/1/EO1H2260862014101110K2_1T/EO1H2260862014101110K2_MTL_L1T.TXT',
 'images/1/LC08_L1TP_226086_20140414_20170423_01_T1/LC08_L1TP_226086_20140414_20170423_01_T1_MTL.txt',
 'images/10/EO1A2240692014363110KF_1T/EO1A2240692014363110KF_MTL_L1T.TXT',
 'images/10/EO1H2240692014363110KF_1T/EO1H2240692014363110KF_MTL_L1T.TXT',
 'images/10/LC08_L1TP_224069_20141228_20170415_01_T1/LC08_L1TP_224069_20141228_20170415_01_T1_MTL.txt',
 'images/11/EO1A1960522014361110KF_1T/EO1A1960522014361110KF_MTL_L1T.TXT',
 'images/11/EO1H1960522014361110KF_1T/EO1H1960522014361110KF_MTL_L1T.TXT',
 'images/11/LC08_L1TP_196052_20141224_20170416_01_T1/LC08_L1TP_196052_20141224_20170416_01_T1_MTL.txt',
 'images/12/EO1A1410452014354110KZ_1T/EO1A1410452014354110KZ_MTL_L1T.TXT',
 'images/12/EO1H1410452014354110KZ_1T/EO1H1410452014354110KZ_MTL_L1T.TXT',
 'images/12/LC08_L1TP_141045_20141223_20170416_01_T1/LC08_L1TP_141045_20141223_20170416_01_

In [7]:
def create_dataset(files):
    corners = ['UL', 'UR', 'LL', 'LR']
    product_corners_eo = ['PRODUCT_{}_CORNER_LAT'.format(corner) for corner in corners] +\
                      ['PRODUCT_{}_CORNER_LON'.format(corner) for corner in corners]
    product_corners_landsat = ['CORNER_{}_LAT_PRODUCT'.format(corner) for corner in corners] +\
                      ['CORNER_{}_LON_PRODUCT'.format(corner) for corner in corners]
    corners_dict = dict(zip(product_corners_eo, product_corners_landsat)) # its needed since the syntax for EO-1 products and Landsat 8 is different for the corners coords
    meta_cols = ['SceneNumber','SPACECRAFT_ID', 'SENSOR_ID', 'ACQUISITION_DATE', 'PRODUCT_ID', 'WRS_PATH', 'WRS_ROW','PRODUCT_TYPE', 'DATUM', 'MAP_PROJECTION'] +\
                    product_corners_eo
    nScene_regex = re.compile('images/([0-9]*)/.*')
    
    df_metadata = pd.DataFrame(columns = meta_cols)
    
    for file in files:
        f = open(file, "r")
        file_str = f.read()
        file_metadata = {}
        file_metadata['SceneNumber'] = nScene_regex.match(file).groups()[0]
        for col in meta_cols:
            pattern = re.compile('.*'+col+'.* = (.*)\\n.*')
            matched = pattern.findall(file_str)
            if len(matched) == 0:
                if col == 'WRS_PATH':
                    matched = re.findall('.*/EO1\w(\d{3})\d{3}.*', file)
                    file_metadata[col] = matched[0]
                if col == 'WRS_ROW':
                    matched = re.findall('.*/EO1\w\d{3}(\d{3}).*', file)
                    file_metadata[col] = matched[0]
                elif col == 'PRODUCT_TYPE':
                    pattern = re.compile('.*DATA_TYPE = (.*)\\n.*')
                    matched = pattern.findall(file_str)
                    file_metadata[col] = matched[0].strip('"')
                elif col == 'DATUM':
                    pattern = re.compile('.*REFERENCE_DATUM = (.*)\\n.*')
                    matched = pattern.findall(file_str)
                    file_metadata[col] = matched[0]
                elif col == 'PRODUCT_ID':
                    matched = re.findall('(EO[A-Z0-9]*)_.*', file)
#                    pattern = re.compile('.*REQUEST_ID = (.*)\\n.*')
#                    matched = pattern.findall(file_str)
                    file_metadata[col] = matched[0]
                elif 'CORNER' in col:
                    file_metadata[col] = re.findall('.*{} = (.*)\\n.*'.format(corners_dict[col]), file_str)[0]
                elif 'DATE' in col:
                    file_metadata[col] = re.findall('.*DATE_ACQUIRED = (.*)\\n.*', file_str)[0]
            else:
                file_metadata[col] = matched[0].strip('"')
        df_metadata = df_metadata.append(file_metadata, ignore_index = True)
        f.close()
    return df_metadata


In [17]:
df_metadata = create_dataset(files_metadata)


IndexError: list index out of range

In [20]:
df_metadata.columns

Index(['SceneNumber', 'SPACECRAFT_ID', 'SENSOR_ID', 'ACQUISITION_DATE',
       'PRODUCT_ID', 'WRS_PATH', 'WRS_ROW', 'PRODUCT_TYPE', 'DATUM',
       'MAP_PROJECTION', 'PRODUCT_UL_CORNER_LAT', 'PRODUCT_UR_CORNER_LAT',
       'PRODUCT_LL_CORNER_LAT', 'PRODUCT_LR_CORNER_LAT',
       'PRODUCT_UL_CORNER_LON', 'PRODUCT_UR_CORNER_LON',
       'PRODUCT_LL_CORNER_LON', 'PRODUCT_LR_CORNER_LON', 'CENTER_LAT',
       'CENTER_LON', 'MODTRAN_ATMOSPHERE'],
      dtype='object')

In [24]:
df_metadata.sort_values(by = 'ACQUISITION_DATE').head(100)

,SceneNumber,SPACECRAFT_ID,SENSOR_ID,ACQUISITION_DATE,PRODUCT_ID,WRS_PATH,WRS_ROW,PRODUCT_TYPE,DATUM,MAP_PROJECTION,...,PRODUCT_UR_CORNER_LAT,PRODUCT_LL_CORNER_LAT,PRODUCT_LR_CORNER_LAT,PRODUCT_UL_CORNER_LON,PRODUCT_UR_CORNER_LON,PRODUCT_LL_CORNER_LON,PRODUCT_LR_CORNER_LON,CENTER_LAT,CENTER_LON,MODTRAN_ATMOSPHERE
0,1,EO1,ALI,2014-04-11 00:00:00+00:00,EO1A2260862014101110K2,226,86,L1T,WGS84,UTM,...,-37.001281,-38.016830,-38.006478,-61.481440,-60.837727,-61.460922,-60.808527,-37.5090,-61.1472,SAS
1,1,EO1,HYPERION,2014-04-11 00:00:00+00:00,EO1H2260862014101110K2,226,86,L1T,WGS84,UTM,...,-37.017285,-37.955235,-37.949705,-61.525054,-61.144115,-61.506598,-61.120898,-37.4862,-61.3242,SAS
2,1,LANDSAT_8,OLI_TIRS,2014-04-14 00:00:00+00:00,LC08_L1TP_226086_20140414_20170423_01_T1,226,86,L1TP,WGS84,UTM,...,-36.376580,-38.562580,-38.517490,-62.430000,-59.791930,-62.413470,-59.699080,-37.4687,-61.0836,SAS
26,2,LANDSAT_8,OLI_TIRS,2014-05-18 00:00:00+00:00,LC08_L1TP_112076_20140518_20170422_01_T1,112,76,L1TP,WGS84,UTM,...,-22.038020,-24.184330,-24.157160,117.663910,119.910140,117.674430,119.956190,-23.1105,118.8012,T
25,2,EO1,HYPERION,2014-05-21 00:00:00+00:00,EO1H1120762014141110KF,112,76,L1T,WGS84,UTM,...,-22.448985,-23.348097,-23.345745,118.114721,118.403303,118.122075,118.412559,-22.8985,118.2632,T
24,2,EO1,ALI,2014-05-21 00:00:00+00:00,EO1A1120762014141110KF,112,76,L1T,WGS84,UTM,...,-22.431988,-23.412843,-23.407147,118.152505,118.755787,118.160787,118.768400,-22.9223,118.4594,T
35,5,LANDSAT_8,OLI_TIRS,2014-07-18 00:00:00+00:00,LC08_L1TP_034020_20140718_20170304_01_T1,34,20,L1TP,WGS84,UTM,...,58.375590,56.200150,56.189170,-100.691600,-96.566740,-100.594080,-96.706930,57.2881,-98.6398,SAS
32,4,LANDSAT_8,OLI_TIRS,2014-07-19 00:00:00+00:00,LC08_L1TP_041029_20140719_20170304_01_T1,41,29,L1TP,WGS84,UTM,...,45.622930,43.562850,43.517160,-116.582740,-113.623720,-116.597580,-113.743530,44.5938,-115.1369,MLS
40,7,EO1,HYPERION,2014-07-20 00:00:00+00:00,EO1H0350292014201110KF,35,29,L1T,WGS84,UTM,...,45.090497,44.205349,44.210271,-106.672214,-106.279687,-106.647113,-106.260474,44.6479,-106.4649,MLS
39,7,EO1,ALI,2014-07-20 00:00:00+00:00,EO1A0350292014201110KF,35,29,L1T,WGS84,UTM,...,45.137799,44.141297,44.149430,-106.620088,-105.838144,-106.592820,-105.824031,44.6395,-106.2188,MLS


In [9]:
df_metadata.dtypes
col_corners = df_metadata.filter(regex = r'CORNER').columns

data_types = {
    'ACQUISITION_DATE': pd.DatetimeTZDtype(tz = 'UTC'),
    'WRS_PATH': 'int32',
    'WRS_ROW': 'int32'}

data_types.update(dict(zip(col_corners, ['float64']*len(col_corners))))

df_metadata = df_metadata.astype(data_types)
df_metadata.dtypes

SceneNumber                           object
SPACECRAFT_ID                         object
SENSOR_ID                             object
ACQUISITION_DATE         datetime64[ns, UTC]
PRODUCT_ID                            object
WRS_PATH                               int32
WRS_ROW                                int32
PRODUCT_TYPE                          object
DATUM                                 object
MAP_PROJECTION                        object
PRODUCT_UL_CORNER_LAT                float64
PRODUCT_UR_CORNER_LAT                float64
PRODUCT_LL_CORNER_LAT                float64
PRODUCT_LR_CORNER_LAT                float64
PRODUCT_UL_CORNER_LON                float64
PRODUCT_UR_CORNER_LON                float64
PRODUCT_LL_CORNER_LON                float64
PRODUCT_LR_CORNER_LON                float64
dtype: object

In [10]:
df_metadata['CENTER_LAT'] = df_metadata.apply(lambda row: np.round(row.filter(regex = 'LAT').values.mean(), 4), axis = 1)
df_metadata['CENTER_LON'] = df_metadata.apply(lambda row: np.round(row.filter(regex = 'LON').values.mean(), 4), axis = 1)

Dentro de la configuración de la correción atmosférica de FLAASH en ENVI, se requiere ingresar el modelo atmosférico de MODTRAN, dentro del manual del módulo de flaash se muestran distintas formas de determinar este modelo, dos de las cuales requieren saber la temperatuar a nivel de superficie del terreno, puesto que no tengo esa información, usaré le tercer método consistente en determinar el modelo según la latitud y el mes del año.

In [11]:
modtran_atm = pd.read_csv('MODTRAN atmospheres.csv', index_col = 'Latitude(°N)')
modtran_atm

,January,February,March,April,May,June,July,August,September,October,November,December
Latitude(°N),,,,,,,,,,,,
80,SAW,SAW,SAW,SAW,SAW,MLW,MLW,MLW,MLW,MLW,SAW,SAW
70,SAW,SAW,SAW,MLW,MLW,MLW,MLW,MLW,MLW,MLW,SAW,SAW
60,MLW,MLW,MLW,MLW,MLW,SAS,SAS,SAS,SAS,SAS,MLW,MLW
50,MLW,MLW,MLW,SAS,SAS,SAS,SAS,SAS,SAS,SAS,SAS,SAS
40,SAS,SAS,SAS,SAS,SAS,MLS,MLS,MLS,MLS,MLS,SAS,SAS
30,MLS,MLS,MLS,MLS,MLS,T,T,T,T,T,MLS,MLS
20,T,T,T,T,T,T,T,T,T,T,T,T
10,T,T,T,T,T,T,T,T,T,T,T,T
0,T,T,T,T,T,T,T,T,T,T,T,T


In [12]:
months_dict = dict((k,v) for k,v in enumerate(calendar.month_name))

df_metadata['MODTRAN_ATMOSPHERE'] = df_metadata.apply(lambda row: modtran_atm.loc[np.round((row.CENTER_LAT),-1).astype(int), months_dict[row.ACQUISITION_DATE.month]], axis = 1)

In [13]:

def create_dataset(save_dataset = False, return_dataset = True, output_path_filename = 'img_metadata.csv'):
   # Variables
    data_types = {
        'ACQUISITION_DATE': pd.DatetimeTZDtype(tz = 'UTC'),
        'WRS_PATH': 'int32',
        'WRS_ROW': 'int32'}

    files_txt = glob.glob(r'images/[0-9]/*/*[.txt|.TXT]')
    mtl_regex = re.compile('(.*/.*MTL.*)')
    files_metadata = []

    corners = ['UL', 'UR', 'LL', 'LR']
    product_corners_eo = ['PRODUCT_{}_CORNER_LAT'.format(corner) for corner in corners] +\
                        ['PRODUCT_{}_CORNER_LON'.format(corner) for corner in corners]
    product_corners_landsat = ['CORNER_{}_LAT_PRODUCT'.format(corner) for corner in corners] +\
                        ['CORNER_{}_LON_PRODUCT'.format(corner) for corner in corners]

    # This dict is needed since the syntax for EO-1 products and Landsat 8 is different for the corners coords
    corners_dict = dict(zip(product_corners_eo, product_corners_landsat)) 

    meta_cols = ['SceneNumber', 'SPACECRAFT_ID', 'SENSOR_ID', 'ACQUISITION_DATE', 'PRODUCT_ID', 'WRS_PATH', 'WRS_ROW','PRODUCT_TYPE',
                'DATUM', 'MAP_PROJECTION'] +  product_corners_eo
    nScene_regex = re.compile('images/([0-9]*)/.*')


    df_metadata = pd.DataFrame(columns = meta_cols)
    
    # File reading
    ## get metadata filenames
    for file_path in files_txt:
        matched = mtl_regex.match(file_path)
        if matched is not None:
            files_metadata.append(matched.group())
    
    ## Read the metadata files of each image
    for file in files_metadata:
        #print('Reading {} ...\n'.format(file))
        f = open(file, "r")
        file_str = f.read()
        file_metadata = {}
        file_metadata['SceneNumber'] = nScene_regex.match(file).groups()[0]

        for col in meta_cols:
            pattern = re.compile(r'.*'+col+'.* = (.*)\\n.*')
            matched = pattern.findall(file_str)
            if len(matched) == 0:
                if col == 'WRS_PATH':
                    matched = re.findall(r'.*/EO1\w(\d{3})\d{3}.*', file)
                    file_metadata[col] = matched[0]
                if col == 'WRS_ROW':
                    matched = re.findall(r'.*/EO1\w\d{3}(\d{3}).*', file)
                    file_metadata[col] = matched[0]
                elif col == 'PRODUCT_TYPE':
                    pattern = re.compile(r'.*DATA_TYPE = (.*)\\n.*')
                    matched = pattern.findall(file_str)
                    file_metadata[col] = matched[0].strip('"')
                elif col == 'DATUM':
                    pattern = re.compile(r'.*REFERENCE_DATUM = (.*)\\n.*')
                    matched = pattern.findall(file_str)
                    file_metadata[col] = matched[0]
                elif col == 'PRODUCT_ID':
                    matched = re.findall(r'(EO[A-Z0-9]*)_.*', file)
                    file_metadata[col] = matched[0]
                elif 'CORNER' in col:
                    file_metadata[col] = re.findall(r'.*{} = (.*)\\n.*'.format(corners_dict[col]), file_str)[0]
                elif 'DATE' in col:
                    file_metadata[col] = re.findall(r'.*DATE_ACQUIRED = (.*)\\n.*', file_str)[0]
            else:
                file_metadata[col] = matched[0].strip('"')
        df_metadata = df_metadata.append(file_metadata, ignore_index = True)
        f.close()

    # Preprocessing code
    #df = create_dataset(files_metadata)
    col_corners = df_metadata.filter(regex = r'CORNER').columns

    data_types.update(dict(zip(col_corners, ['float64']*len(col_corners))))

    # Center of the image
    df_metadata = df_metadata.astype(data_types)
    df_metadata['CENTER_LAT'] = df_metadata.apply(lambda row: np.round(row.filter(regex = 'LAT').values.mean(), 4), axis = 1)
    df_metadata['CENTER_LON'] = df_metadata.apply(lambda row: np.round(row.filter(regex = 'LON').values.mean(), 4), axis = 1)

    '''
    In the atmospheric correction of FLAASH in ENVI, its required the MODTRAN model. On the FLAASH user guide various methods are showed for
    determining the corresponding atmospheric modelo to use. This script will calculate the model based in the latitude of the center point
    of the image and the month of the year.
    '''

    months_dict = dict((k,v) for k,v in enumerate(calendar.month_name))
    modtran_atm = pd.read_csv('MODTRAN atmospheres.csv', index_col = 'Latitude(°N)')

    df_metadata['MODTRAN_ATMOSPHERE'] = df_metadata.apply(lambda row: modtran_atm.loc[
                                                                np.round((row.CENTER_LAT),-1).astype(int),
                                                                months_dict[row.ACQUISITION_DATE.month]],
                                                                axis = 1)

    if save_dataset is True:
        print('save')
        df_metadata.to_csv(output_path_filename, index = False)
    if return_dataset is True:
        return df_metadata



In [14]:
#from data_preproc import create_dataset
df_test = create_dataset(return_dataset=True, save_dataset=True)
df_test

IndexError: list index out of range

In [16]:
import numpy as np
modtran_atm.loc[np.round((-37.5090),-1).astype(int), 'April']

'SAS'

----------------

In [2]:
import requests
from usgs import api

In [7]:
api.login('ignacio.loayza', '4YApm7M32z3i4pm', catalogId='EE')

{'errorCode': None,
 'error': '',
 'data': '181cf2b59bb84ce9a19d1513ffc5e495',
 'api_version': '1.4.0',
 'access_level': 'user',
 'catalog_id': 'default',
 'executionTime': 0.40836596488952637}

In [9]:
api.search('LANDSAT_8_C1', 'EE', start_date='2017-04-01', end_date='2017-05-01', max_results=10, extended=True)

{'errorCode': None,
 'error': '',
 'data': {'numberReturned': 10,
  'totalHits': 20399,
  'firstRecord': 1,
  'lastRecord': 10,
  'nextRecord': 11,
  'results': [{'acquisitionDate': '2017-05-01',
    'startTime': '2017-05-01',
    'endTime': '2017-05-01',
    'spatialFootprint': {'type': 'Polygon',
     'coordinates': [[[-22.07974, 80.13956],
       [-16.19818, 78.78363],
       [-8.20143, 79.71156],
       [-13.5208, 81.19994],
       [-22.07974, 80.13956]]]},
    'sceneBounds': '-22.07974,78.78363,-8.20143,81.19994',
    'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_8_c1/2017/008/002/LC08_L1TP_008002_20170501_20170515_01_T1.jpg',
    'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&ordered=LC80080022017121LGN00&node=INVSVC',
    'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC80080022017121LGN00/INVSVC/',
    'entityId': 'LC80080022017121LGN00',
    'displayId': 'LC08_L1TP_008002_20170501_20170515_